# Text2SPARQL

This is a development workbook for getting the hang of training models.

In [1]:
import os
import torch

os.environ["CUDA_VISIBLE_DEVICES"]="1"
print(f"CUDA Available: {torch.cuda.is_available()}")

CUDA Available: True


In [2]:
DATASET_NAME = "lcquad2"
DATASET_FOLDER = "data"
DATASET_PATH = os.path.join(DATASET_FOLDER, DATASET_NAME)

ACCELERATE_USE = False 
ACCELERATE_STR = "-accelerate" if ACCELERATE_USE else ""

MODEL_NAME = "t5-small" # With t5-small, the non accelerated training works better than accelerated?
MODEL_TYPE = "text2sparql"
MODEL_FULL = f"{MODEL_TYPE}-{MODEL_NAME}-{DATASET_NAME}{ACCELERATE_STR}"

MODEL_FOLDER = "models"
MODEL_PATH = os.path.join(MODEL_FOLDER, MODEL_FULL)

EVALUATION_FOLDER = os.path.join(MODEL_PATH, "evaluations")
CHECKPOINT_FOLDER = os.path.join(MODEL_PATH, "checkpoints")

folders = [MODEL_FOLDER, EVALUATION_FOLDER, CHECKPOINT_FOLDER]

In [3]:
assert os.path.exists(DATASET_PATH)

for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

## Preprocessing

Bannerjee does some preprocessing of the LCQuAD dataset,
I try to replicate that here.

First we load some files into memory

In [4]:
import json
import pickle
import os
from os.path import join
from pprint import pprint

assert DATASET_PATH.endswith("lcquad2")
lcquad2_dir = DATASET_PATH

# LCQuAD2 entity labels
with open(join(lcquad2_dir, "lcq2_labels.pickle"), "rb") as f:
    labels = pickle.load(f)

pprint("# Sample Entity Labels")
pprint(list(labels[k] for k in ['q51366', 'q15779', 'q23906217']))
print()

# Training Data has exactly the same file size as the official one
with open(join(lcquad2_dir, "train.json")) as f:
    data = json.load(f)

pprint("# Sample Question, Query")
pprint(list(data[1][k] for k in ["question", "sparql_wikidata"]))
print()

# Load the relation labels
with open(join(lcquad2_dir, "relations.json")) as f:
    rel_labels = json.load(f)

pprint("# Sample Relation Labels")
pprint(list(rel_labels[k] for k in ["P10", "P6"]))
print()

# Load the sparql vocabulary
with open(join(lcquad2_dir, "vocab.txt")) as f:
    vocab = list(map(lambda x: x.strip(), f.readlines()))
    vocab.append('null') # not too sure what this is for

pprint("# Sample SPARQL Vocab")
pprint(vocab[1:5])
print()

'# Sample Entity Labels'
['Chandrasekhar limit', 'toluene', 'Olympic victor, stadion']



'# Sample Question, Query'
["Who is the child of Ranavalona I's husband?",
 'SELECT ?answer WHERE { wd:Q169794 wdt:P26 ?X . ?X wdt:P22 ?answer}']

'# Sample Relation Labels'
['video', 'head of government']

'# Sample SPARQL Vocab'
['(', 'rdfs:label', 'by', 'ask']



Some labels are missing from the lcq2_labels.pickle,
and cause run time errors in the script.
We add them back here to avoid this problem
(though ideally we should find a better label to entity map)

In [5]:
labels['quercia']='null'
labels['qui']='null'
labels['}']='null'
labels['p5122'] = 'Ontario public library ID'.lower()
labels['p3888']='Boijmans artist ID'
labels['p5388']='Bulgarian Antarctic Gazetteer ID'
labels['p5151']='Israel Film Fund ID'
labels['p3633']='British Museum place ID'
labels['p1733']='Steam application ID'

Next we assign vocabularies to tokens.

In [6]:
vocab_dict = {}
for i, text in enumerate(vocab):
    vocab_dict[text] = f'<extra_id_{i}>'

pprint("# Sample Masked Tokens")
pprint([vocab_dict[k] for k in ['"', 'null', '?value']])
print()

'# Sample Masked Tokens'
['<extra_id_0>', '<extra_id_60>', '<extra_id_16>']



And adjust some labels to use the null token

In [7]:
for k in labels:
    if labels[k] is None:
        labels[k] = vocab_dict['null']
        # print(f'{k}: {labels[k]}')


## Some Useful Functions

In [8]:
def xprint(thing):
    pprint(thing)
    return thing

def compare(x, y=None):

    def _compare(z):
        pprint(f"Old: {x}")
        pprint(f"New: {z}")
    
    if not y:
        return lambda z : _compare(z)
    else:
        return lambda : _compare(y)

Now we reformate the dataset
- Note: it seems that Bannerjee replaces training data
that has no questions with the Natural Language version.

For reference these are the definition of each feature,
taken **verbatim** from their [homepage](https://sda.tech/projects/lc-quad-2/)
```
{
     "uid": a unique id number
     "sparql_wikidata": a sparql fro wikidata endpoint
     "sparql_dbpedia18": a sparql for DBpedia endpoint which has wikidata information
     "NNQT_question": system generated question,
     "question": Verbalised question,
     "paraphrased_question": paraphrased version of the verbalised question,
     "template_id": id for the template
     "template": template discription    
}
```

In [9]:
import re

data_x, data_y = [], []
data_x_shuffle = []

for i, inst in enumerate(data):
    wikisparql = inst['sparql_wikidata']
    if inst['question'] is None:
        question = inst['NNQT_question']
    else:
        question = inst['question']
    question = question.replace('{', '').replace('}', '')

    match_str = r"\'(.*?)\'"
    hashi = {}
    # To mask filter literals
    if re.search(match_str, wikisparql):
        lits=re.findall(match_str,wikisparql)
        # print(f"Old: {wikisparql}")
        for j, lit in enumerate(lits):
            idx = j + 1
            wikisparql = wikisparql.replace(f"'{lit.strip()}'", f"'###{idx}'")
            hashi[f'###{idx}'] = lit.strip()
        # print(f"New: {wikisparql}")
    
    # there is an extra space beacuse of http: and https:
    sparql = wikisparql.replace('(',' ( ').replace(')',' ) ') \
    .replace('{',' { '). \
    replace('}',' } ').replace('wd:','wd: ').replace('wdt:','wdt: '). \
    replace(' p:',' p: ').replace(' ps:',' ps: ').replace('pq:','pq: '). \
    replace(',',' , ').replace(",'",", '").replace("'"," ' ").replace('.',' . '). \
    replace('=',' = ').replace('  ',' ').lower()
    
    # print(f"sparql: {sparql}")
    # select distinct ?obj where { wd: q188920 wdt: p2813 ?obj . ?obj wdt: p31 wd: q1002697 } 

    _ents = re.findall( r'wd: (?:.*?) ', sparql) # ['wd: q188920 ', 'wd: q1002697 ']
    _ents_for_labels = re.findall( r'wd: (.*?) ', sparql) # ['q188920', 'q1002697']
    
    _rels = re.findall( r'wdt: (?:.*?) ',sparql)
    _rels += re.findall( r' p: (?:.*?) ',sparql)
    _rels += re.findall( r' ps: (?:.*?) ',sparql)
    _rels += re.findall( r'pq: (?:.*?) ',sparql) # ['wdt: p2813 ', 'wdt: p31 ']
    # Missing rdfs:label, not sure if that is important
    
    _rels_for_labels = re.findall( r'wdt: (.*?) ',sparql)
    _rels_for_labels += re.findall( r' p: (.*?) ',sparql)
    _rels_for_labels += re.findall( r' ps: (.*?) ',sparql)
    _rels_for_labels += re.findall( r'pq: (.*?) ',sparql) # ['p2813', 'p31']

    # print(_rels)
    # print(_rels_for_labels)
    for j in range(len(_ents_for_labels)):
        # print('Q'+_ents_for_labels[j][1:])
        if '}' in _ents[j]: # Entry 12686 is malformed
            # pprint(inst)
            # pprint(_ents)
            _ents[j]=''
        _ents[j]=_ents[j]+labels[_ents_for_labels[j]]+' '
        # wd: q36970 -> wd: q36970 Jackie Chan

    for j in range(len(_rels_for_labels)):
        if _rels_for_labels[j].upper() not in rel_labels:
            # For some reasons the original preprocess.py didnt convert to upper?
            rel_labels['P'+_rels_for_labels[j][1:]]=vocab_dict['null']
        _rels[j]=_rels[j]+rel_labels['P'+_rels_for_labels[j][1:]]+' '
        # wdt: p26 -> wdt: p26 spouse
    # print(_ents)

    _ents+=_rels
    # random.shuffle(_ents)
    # random.shuffle(_rels)

    # move to a function
    newvars = ['?vr0','?vr1','?vr2','?vr3','?vr4','?vr5']
    sparql_split = sparql.split()
    variables = set([x for x in sparql_split if x[0] == '?'])
    for j, var in enumerate(sorted(variables)):
        if var == '?maskvar1': #???
            print(sparql)
            continue
        sparql = sparql.replace(var, newvars[j]) # Normalize var names
    
    # old = compare(sparql)

    split = sparql.split()
    
    for j, item in enumerate(split):
        if item in vocab_dict:
            split[j] = vocab_dict[item]
    
    split = ' '.join(split).strip()
    # old(split)

    for keys in hashi:
        split = split.replace(keys, hashi[keys])
    
    data_y.append(split)

    for rel in _ents:
        rel=rel.replace('wd:',vocab_dict['wd:']+' ')
        rel=rel.replace('wdt:',vocab_dict['wdt:']+' ')
        old = compare(rel)
        if 'p:' in rel:
            if 'http' in rel:
                print(inst) # There are no more http
            rel=rel.replace('p:',vocab_dict['p:']+' ')
            # old(rel)
        rel=rel.replace('ps:',vocab_dict['ps:']+' ')
        rel=rel.replace('pq:',vocab_dict['pq:']+' ')
        question=question+' '+vocab_dict['[DEF]']+' '+rel # When question gets masked
    data_x.append(question.strip())

assert len(data_x) == len(data_y)

Now we need to save the data.

In [10]:
import pandas as pd
df = pd.DataFrame({
    'x': data_x,
    'y': data_y,
    })

save_file = join(lcquad2_dir, 'preprocessed_data.csv')
df.to_csv(save_file)

In [11]:
print(len(df))
df.head()

24180


,x,y
0,What periodical literature does Delta Air Line...,<extra_id_6> <extra_id_21> <extra_id_39> <extr...
1,Who is the child of Ranavalona I's husband? <e...,<extra_id_6> <extra_id_39> <extra_id_19> <extr...
2,Is it true Jeff_Bridges occupation Lane Chandl...,<extra_id_4> <extra_id_19> <extra_id_33> <extr...
3,What is the pre-requisite of phase matter of G...,<extra_id_6> <extra_id_39> <extra_id_19> <extr...
4,Which is the operating income for Qantas? <ext...,<extra_id_6> <extra_id_21> <extra_id_39> <extr...


# Model

Now we need to generate a T5 model for fine tuning

In [12]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import transformers
# from accelerate import init_empty_weights, dispatch_model, infer_auto_device_map, load_checkpoint_and_dispatch
from huggingface_hub import hf_hub_download
import torch
import torch.nn as nn
import torch.optim as optim
import random
import math

model_name = MODEL_NAME # "t5-small"

class Model(nn.Module):
    def __init__(self, model_name):
        super(Model, self).__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto")

        pprint("# Model Device Map")
        pprint(self.model.hf_device_map)
        print()
    
    def forward(self, input):
        outputs = self.model(
            input_ids = input['input_ids'],
            labels = input['labels'],
            attention_mask = input['attention_mask'],
            output_hidden_states = True,
            output_attentions = True
        )

        return outputs.loss

# model = T5ForConditionalGeneration.from_pretrained(model_name, device_map="auto") # Device_map splits the load over multiple GPUs, this seems to be quite new

And the Trainer

In [13]:
import tqdm

class Train:
    def __init__(self,data,data_val, model_name):
        self.data=data
        self.dev_data=data_val

        self.tokenizer=T5Tokenizer.from_pretrained(model_name)
        self.model=Model(model_name)
        # self.model.to(f'cuda:{self.model.device_ids[0]}')  
           
        # Modify lr?
        self.optimizer=optim.AdamW(self.model.parameters(),lr=0.0015)
        self.lr_scheduler=transformers. \
        get_polynomial_decay_schedule_with_warmup(self.optimizer, 5000, 30000,power=0.5)

        self.iters=60000
        self.print_every=100
        self.eval_every=8000
        # self.num_gpus=1
        self.eval_bs=6
        self.bs=5
        self.back_propogate=10
        
        self.train()

    def generate_batch(self):
        output=random.sample(self.data,self.bs)
        inp,label=[],[]
        for dat in output:
            inp.append(dat[0])
            label.append(dat[1])

        return inp,label

    def preprocess_function(self,inputs, targets):
        model_inputs=self.tokenizer(inputs, padding=True, \
                        return_tensors='pt',max_length=512, truncation=True)
        labels=self.tokenizer(targets,padding=True,max_length=512, truncation=True)

        if True:
            labels["input_ids"] = [
            [(l if l != self.tokenizer.pad_token_id else -100) \
             for l in label] for label in labels["input_ids"]
            ]
        labels['input_ids']=torch.tensor(labels['input_ids'])
        model_inputs["labels"]=labels["input_ids"].to(0)
        model_inputs["input_ids"]=model_inputs["input_ids"].to(0)
        model_inputs["attention_mask"]=model_inputs["attention_mask"].to(0)

        return model_inputs

    def val(self,o):
        pprint('# Validating...')
        self.model.eval()
        acc,bs,i=0,self.eval_bs,0
        saver=[]

        # progress_bar = tqdm.auto.tqdm(range(math.ceil(len(self.dev_data) / bs)))
        # progress_bar.set_description(f"Eval {o}")
           
        while i<len(self.dev_data):
            bs_=min(bs,len(self.dev_data)-i)
            if i % (100) == 0:
                print(f"Evaluation {i}/{len(self.dev_data)}")
            i+=bs_
            inp,label=[],[]
            for j in range(i-bs_,i):
                inp.append(self.dev_data[j][0])
                label.append(self.dev_data[j][1])
            

            input=self.preprocess_function(inp,label)

            output=self.model.model.generate(input_ids=input['input_ids'],
                      num_beams=10,attention_mask=input['attention_mask'], \
                        early_stopping=True, max_length=200,output_hidden_states=True,output_attentions=True)
            
            out=self.tokenizer.batch_decode(output,skip_special_tokens=False)

            for k in range(len(out)):
                #print(out[k].replace('<pad>','').replace('</s>','').strip())
                a1=out[k].replace('<pad>','').replace('</s>','').replace('<unk>','').replace('<s>','').strip().replace(' ','')
                a2=label[k].strip().replace(' ','')
                # print(a1, '       ', a2)
                saver.append({'input':inp[k],'gold':label[k].strip(),'generated':out[k].replace('<pad>',''). \
                      replace('</s>','').replace('<unk>','').replace('<s>','').strip()})
                if a1==a2:
                    acc+=1; #print('ttt')

            # progress_bar.update(1)
        
        file=open(join(EVALUATION_FOLDER, 'dev_result_'+str(o)+'.json'),'w')
        json.dump(saver,file)
        pprint(f'# Saved {file.name}')
        file.close()
        return 100*acc/len(self.dev_data)

    def train(self):

        scalar=0
        for i in range(self.iters):
            self.model.train()
            inp,label=self.generate_batch()
            input=self.preprocess_function(inp,label)
            loss=self.model(input)

            scalar+=loss.mean().item()
            if(i+1)%self.print_every==0:
                print('iteration={}, training loss={}'.format(i+1,scalar/self.print_every))
                scalar=0
            if(i + 1)%self.eval_every==0:
                acc=self.val(i+1)
                print('validation acc={}'.format(acc))

                torch.save(self.model.state_dict(),
                       join(CHECKPOINT_FOLDER,'checkpoint_'+str(i + 1)+'.pth'))
                
            loss/=self.back_propogate
            loss.mean().backward()
            if (i+1)%self.back_propogate:
                self.optimizer.step();
                self.lr_scheduler.step();
                self.optimizer.zero_grad()


In [14]:
len(df.values.tolist())*0.7

16926.0

In [15]:
data = df.values.tolist()
total_len = len(data)
split_len = round(total_len*0.3)
final_data_dev, final_data = data[:split_len], data[split_len:]
pprint("# Beginning training")
trainer = Train(final_data, final_data_dev, MODEL_NAME)

'# Beginning training'


'# Model Device Map'
{'': 0}



iteration=100, training loss=11.931724805831909


iteration=200, training loss=8.55273416042328


iteration=300, training loss=5.0989305424690246


...

iteration=59600, training loss=0.009473071200627601


iteration=59700, training loss=0.010281414847631822


iteration=59800, training loss=0.009376118330983445


iteration=59900, training loss=0.008951069866889157


iteration=60000, training loss=0.010690901361231226
